# Own Covariance function for the Heat Equation

In [1]:
# Unfortunately the kernel is basically equal to zero and the optimizer will stick with the default value
# for the hyperparameter, which in turn returns 1.0 for phi.

# Own Kernel function:
class MyKernel2(Kernel):

    def __init__(self, log_phi=0.):
        self.hyp = [log_phi]
        
    def getCovMatrix(self,x=None,z=None,mode=None):
        self.checkInputGetCovMatrix(x,z,mode)
        p = np.exp(self.hyp[0])           # phi
        A = 0
        if not x is None:
            n, D = x.shape
            A = np.zeros((n,n))
        if not z is None:
            nn, D = z.shape
            A = np.zeros((nn,nn))
        if mode == 'self_test':
            A = np.zeros((nn,1))
        elif mode == 'train':             # compute covariance matrix for training set
            A = np.zeros((n,n))
            for i in range(n):
                for j in range(n):
                    A[i][j] = (p**2*(7.36e-33*(x[i][1] - x[j][1])**4 - 3.74e-26*(x[i][1] - x[j][1])**2 + 1.58e-20) - 5.28e-21*(x[i][0] - x[j][0])**2 + 4.47e-15)*np.exp(-5.9e-7*(x[i][0] - x[j][0])**2 - 5.9e-7*(x[i][1] - x[j][1])**2)
        elif mode == 'cross':             # compute covariance between data sets x and z
            m = z.shape[0]
            A = np.zeros((n,m))
            for i in range(n):
                for j in range(m):
                    A[i][j] = (p**2*(0.67*(x[i][1] - z[j][1])**4 - 20.75*(x[i][1] - z[j][1])**2 + 53.61) - 17.87*(x[i][0] - z[j][0])**2 + 92.34)*np.exp(-0.1*(x[i][0] - z[j][0])**2 - 0.1*(x[i][1] - z[j][1])**2)
        return A
        
# We are taking the derivative w.r.t. p, but are multiplying it with 2*p or p, since that seems to be the pattern
# in the source code of pyGPs as well:
    def getDerMatrix(self,x=None,z=None,mode=None,der=None):
        self.checkInputGetCovMatrix(x,z,mode)
        p = np.exp(self.hyp[0])           # phi
        n = 0
        if not x is None:
            n, D = x.shape
        if not z is None:
            nn, D = z.shape
        if mode == 'self_test':           # self covariances for the test cases
            A = np.zeros((nn,1))
        elif mode == 'train':             # compute covariance matix for dataset x
            A = np.zeros((n,n))
            for i in range(n):
                for j in range(n):
                    A[i][j] = (p**2*(7.36e-33*(x[i][1] - x[j][1])**4 - 3.74e-26*(x[i][1] - x[j][1])**2 + 1.58e-20) - 5.28e-21*(x[i][0] - x[j][0])**2 + 4.47e-15)*np.exp(-5.9e-7*(x[i][0] - x[j][0])**2 - 5.9e-7*(x[i][1] - x[j][1])**2)
        elif mode == 'cross':             # compute covariance between data sets x and z
            A = np.zeros((n,nn))
            for i in range(n):
                for j in range(nn):
                    A[i][j] = (p**2*(0.67*(x[i][1] - z[j][1])**4 - 20.75*(x[i][1] - z[j][1])**2 + 53.61) - 17.87*(x[i][0] - z[j][0])**2 + 92.34)*np.exp(-0.1*(x[i][0] - z[j][0])**2 - 0.1*(x[i][1] - z[j][1])**2)          
        if der == 0:    # compute derivative matrix wrt 1st parameter
            if mode == 'train':
                A = np.zeros((n,n))
                for i in range(n):
                    for j in range(n):
                        A[i][j] = 4*p**2*(7.36e-33*(x[i][1] - x[j][1])**4 - 3.74e-26*(x[i][1] - x[j][1])**2 + 1.58e-20)*np.exp(-5.9e-7*(x[i][0] - x[j][0])**2 - 5.9e-7*(x[i][1] - x[j][1])**2)
            elif mode == 'cross':
                A = np.zeros((n,nn))
                for i in range(n):
                    for j in range(nn):
                        A[i][j] = 4*p**2*(0.67*(x[i][1] - z[j][1])**4 - 20.75*(x[i][1] - z[j][1])**2 + 53.61)*np.exp(-0.1*(x[i][0] - z[j][0])**2 - 0.1*(x[i][1] - z[j][1])**2)
        else:
            raise Exception("Calling for a derivative in RBF that does not exist")
        return A

NameError: name 'Kernel' is not defined

In [ ]:
# Has to be added to pyGPs/Testing/unit_test_cov.py and then unit_test_cov.py has to be executed.
# For testing purposes only:
def test_cov_new(self):
    k = pyGPs.cov.MyKernel()     # specify your covariance function
    self.checkCovariance(k)
    